# Lesson trading with AI

In [119]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import datetime
from matplotlib import pyplot as plt
import statsmodels.api as sm    
import pandas as pd 
import numpy as np
import yfinance as yf
import pandas_ta
import warnings

warnings.filterwarnings("ignore")
sp500  = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")[0]
sp500['sysbol'] = sp500['Symbol'].str.replace('.','-')
symbols_list = sp500['Symbol'].unique().tolist()
end_date = "2024-01-01"
start_date = pd.to_datetime(end_date) - pd.DateOffset(years=8)

df = yf.download(tickers = symbols_list, start = start_date, end = end_date).stack()
df.index.names = ['Date','Ticker']
df.columns = df.columns.str.lower()
df



[*********************100%***********************]  503 of 503 completed

6 Failed downloads:
['GEV', 'AMTM', 'SW', 'SOLV']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2016-01-01 00:00:00 -> 2024-01-01) (Yahoo error = "Data doesn\'t exist for startDate = 1451624400, endDate = 1704085200")')
['BRK.B']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')
['BF.B']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2016-01-01 00:00:00 -> 2024-01-01)')


Price                              adj close       close        high  \
Date                      Ticker                                       
2016-01-04 00:00:00+00:00 A        37.941662   40.689999   41.189999   
                          AAPL     23.886835   26.337500   26.342501   
                          ABBV     39.654160   57.610001   58.320000   
                          ABT      36.403992   42.930000   44.049999   
                          ACGL     22.950001   22.950001   23.219999   
...                                      ...         ...         ...   
2023-12-29 00:00:00+00:00 XYL     113.428398  114.360001  114.680000   
                          YUM     128.731277  130.660004  131.250000   
                          ZBH     120.933403  121.699997  122.400002   
                          ZBRA    273.329987  273.329987  276.309998   
                          ZTS     195.897491  197.369995  198.009995   

Price                                    low        open       volume  
Date                      Ticker                                       
2016-01-04 00:00:00+00:00 A        40.340000   41.060001    3287300.0  
                          AAPL     25.500000   25.652500  270597600.0  
                          ABBV     56.400002   58.060001   10308400.0  
                          ABT      42.500000   43.939999   12227500.0  
                          ACGL     22.616667   22.940001    1220700.0  
...                                      ...         ...          ...  
2023-12-29 00:00:00+00:00 XYL     113.930000  114.089996     698900.0  
                          YUM     130.210007  130.410004    1196800.0  
                          ZBH     121.239998  121.459999     849600.0  
                          ZBRA    272.769989  274.730011     251300.0  
                          ZTS     196.250000  196.679993    1007200.0  

[980029 rows x 6 columns]

In [120]:
df['garman_klass_vol'] = ((np.log(df['high']) - np.log(df['low']))**2)/2 - (2*np.log(2)-1) * ((np.log(df['adj close']) - np.log(df['open']))**2)
df['rsi'] =  df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.rsi(x, length=20))

def calc_bbands_low(x):
    log_x = np.log1p(x)
    bb = pandas_ta.bbands(close=log_x, length=20)
    if bb is not None:
        return bb.iloc[:,0].set_axis(x.index)
    else:
        return pd.Series([np.nan] * len(x), index=x.index)
    
def calc_bbands_mid(x):
    log_x = np.log1p(x)
    bb = pandas_ta.bbands(close=log_x, length=20)
    if bb is not None:
        return bb.iloc[:,1].set_axis(x.index)
    else:
        return pd.Series([np.nan] * len(x), index=x.index)
    
def calc_bbands_high(x):
    log_x = np.log1p(x)
    bb = pandas_ta.bbands(close=log_x, length=20)
    if bb is not None:
        return bb.iloc[:,2].set_axis(x.index)
    else:
        return pd.Series([np.nan] * len(x), index=x.index)        

def compute_atr(stock_data):
    atr = pandas_ta.atr(high=stock_data['high'], low=stock_data['low'], close=stock_data['close'], length=14)
    return atr.sub(atr.mean()).div(atr.std())

def compute_macd(close):
    macd = pandas_ta.macd(close=close, length=20).iloc[:,0]
    return macd.sub(macd.mean()).div(macd.std())

# pandas_ta.bbands(close=df.xs('AAPL', level=1)['close'], length=20)
df['bb-low'] = df.groupby(level=0)['adj close'].apply(calc_bbands_low).reset_index(level=0,drop=True)
df['bb-mid'] = df.groupby(level=0)['adj close'].apply(calc_bbands_mid).reset_index(level=0,drop=True)
df['bb-high'] = df.groupby(level=0)['adj close'].apply(calc_bbands_high).reset_index(level=0,drop=True)
df['atr'] = df.groupby(level=1, group_keys=False).apply(compute_atr)
df['macd'] = df.groupby(level=1, group_keys=False)['adj close'].apply(compute_macd)
df['dollar_volume'] = (df['adj close']*df['volume'])/1e6
# df['bb-low'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20)).iloc[:,0] # it's old one 
# df['bb-mid'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,1])  # this too 
# df['bb-high'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,2]) # this too 
df

Price                              adj close       close        high  \
Date                      Ticker                                       
2016-01-04 00:00:00+00:00 A        37.941662   40.689999   41.189999   
                          AAPL     23.886835   26.337500   26.342501   
                          ABBV     39.654160   57.610001   58.320000   
                          ABT      36.403992   42.930000   44.049999   
                          ACGL     22.950001   22.950001   23.219999   
...                                      ...         ...         ...   
2023-12-29 00:00:00+00:00 XYL     113.428398  114.360001  114.680000   
                          YUM     128.731277  130.660004  131.250000   
                          ZBH     120.933403  121.699997  122.400002   
                          ZBRA    273.329987  273.329987  276.309998   
                          ZTS     195.897491  197.369995  198.009995   

Price                                    low        open       volume  \
Date                      Ticker                                        
2016-01-04 00:00:00+00:00 A        40.340000   41.060001    3287300.0   
                          AAPL     25.500000   25.652500  270597600.0   
                          ABBV     56.400002   58.060001   10308400.0   
                          ABT      42.500000   43.939999   12227500.0   
                          ACGL     22.616667   22.940001    1220700.0   
...                                      ...         ...          ...   
2023-12-29 00:00:00+00:00 XYL     113.930000  114.089996     698900.0   
                          YUM     130.210007  130.410004    1196800.0   
                          ZBH     121.239998  121.459999     849600.0   
                          ZBRA    272.769989  274.730011     251300.0   
                          ZTS     196.250000  196.679993    1007200.0   

Price                             garman_klass_vol        rsi    bb-low  \
Date                      Ticker                                          
2016-01-04 00:00:00+00:00 A              -0.002193        NaN       NaN   
                          AAPL           -0.001436        NaN       NaN   
                          ABBV           -0.055597        NaN       NaN   
                          ABT            -0.013033        NaN       NaN   
                          ACGL            0.000346        NaN       NaN   
...                                            ...        ...       ...   
2023-12-29 00:00:00+00:00 XYL             0.000008  75.881247  2.635991   
                          YUM            -0.000033  59.338327  2.714815   
                          ZBH             0.000038  68.767222  2.715016   
                          ZBRA            0.000073  67.914673  2.736016   
                          ZTS             0.000034  66.791321  2.874206   

Price                               bb-mid   bb-high       atr      macd  \
Date                      Ticker                                           
2016-01-04 00:00:00+00:00 A            NaN       NaN       NaN       NaN   
                          AAPL         NaN       NaN       NaN       NaN   
                          ABBV         NaN       NaN       NaN       NaN   
                          ABT          NaN       NaN       NaN       NaN   
                          ACGL         NaN       NaN       NaN       NaN   
...                                    ...       ...       ...       ...   
2023-12-29 00:00:00+00:00 XYL     4.311145  5.986298 -0.345889  1.965728   
                          YUM     4.374012  6.033209 -0.073239  0.784239   
                          ZBH     4.371818  6.028621 -1.054422  0.973531   
                          ZBRA    4.362546  5.989075 -0.154931  1.577769   
                          ZTS     4.464028  6.053849  0.420520  2.164720   

Price                             dollar_volume  
Date                      Ticker                 
2016-01-04 00:00:00+00:00 A          124.725625 

In [121]:
last_cols = [c for c in df.columns.unique(0) if c not in ['dollar_volume','volume','open','high','low','close']]
data = (pd.concat([df.unstack('Ticker')['dollar_volume'].resample('M').mean().stack('Ticker').to_frame('dollar_volume'),df.unstack()[last_cols].resample('M').last().stack('Ticker')],axis=1)).dropna()
data
# df.unstack('Ticker')['dollar_volume'].resample('M').mean().stack('Ticker').to_frame('dollar_volume').reset_index()

dollar_volume   adj close  garman_klass_vol  \
Date                      Ticker                                                
2016-02-29 00:00:00+00:00 ALB         68.551473   50.216415         -0.005011   
                          ALGN        41.078421   66.029999          0.000114   
                          ALL        169.004223   52.419048         -0.015592   
                          ALLE        68.485654   57.120575         -0.004677   
                          AMAT       213.800253   17.097706         -0.004035   
...                                         ...         ...               ...   
2023-12-31 00:00:00+00:00 PLTR       993.120310   17.170000          0.000326   
                          CEG        216.240947  116.266106          0.000060   
                          GEHC       172.074635   77.233734          0.000051   
                          KVUE       452.331382   20.884251         -0.000181   
                          VLTO       151.811671   82.034897          0.000128   

                                        rsi    bb-low    bb-mid   bb-high  \
Date                      Ticker                                            
2016-02-29 00:00:00+00:00 ALB     59.740283  2.634789  3.677094  4.719399   
                          ALGN    57.221387  2.642438  3.708415  4.774392   
                          ALL     58.808412  2.711919  3.750457  4.788995   
                          ALLE    58.381560  2.724042  3.770426  4.816809   
                          AMAT    62.166876  2.628778  3.738811  4.848844   
...                                     ...       ...       ...       ...   
2023-12-31 00:00:00+00:00 PLTR    44.949231  2.540490  4.605092  6.669694   
                          CEG     49.455869  3.089159  4.582949  6.076739   
                          GEHC    64.270051  2.722583  4.604533  6.486483   
                          KVUE    60.482250  2.333034  4.166136  5.999238   
                          VLTO    63.031254  3.089745  4.901492  6.713238   

                                       atr      macd  
Date                      Ticker                      
2016-02-29 00:00:00+00:00 ALB    -0.811681  0.187783  
                          ALGN   -1.337798  0.009129  
                          ALL    -0.938187  0.534520  
                          ALLE   -0.225685  0.327839  
                          AMAT   -1.136719 -0.002914  
...                                    ...       ...  
2023-12-31 00:00:00+00:00 PLTR   -0.464274 -0.377428  
                          CEG     0.396988 -0.775277  
                          GEHC   -1.107846  1.002127  
                          KVUE   -1.401678  2.116213  
                          VLTO   -1.002239  1.367596  

[44465 rows x 9 columns]

In [122]:
data['dollar_volume'] = data['dollar_volume'].unstack('Ticker').rolling(window=5*12,min_periods=1).mean().stack()
data['dollar_vol_rank'] = data.groupby('Date')['dollar_volume'].rank(ascending=False)
data = data[data['dollar_vol_rank'] < 150].drop(['dollar_volume', 'dollar_vol_rank'], axis=1)
data

adj close  garman_klass_vol        rsi  \
Date                      Ticker                                            
2016-02-29 00:00:00+00:00 AMAT     17.097706         -0.004035  62.166876   
                          AMGN    110.771690         -0.030163  40.000752   
                          AMT      75.906075         -0.010815  54.138667   
                          AMZN     27.625999          0.000240  47.406555   
                          APTV     53.478298         -0.000589  48.356096   
...                                      ...               ...        ...   
2023-12-31 00:00:00+00:00 MRNA     99.449997          0.000363  65.939985   
                          UBER     61.570000          0.000375  64.522175   
                          CRWD    255.320007          0.000215  73.393415   
                          PLTR     17.170000          0.000326  44.949231   
                          KVUE     20.884251         -0.000181  60.482250   

                                    bb-low    bb-mid   bb-high       atr  \
Date                      Ticker                                           
2016-02-29 00:00:00+00:00 AMAT    2.628778  3.738811  4.848844 -1.136719   
                          AMGN    1.804379  3.522658  5.240937 -0.105060   
                          AMT     1.820192  3.571928  5.323663 -0.780094   
                          AMZN    1.796853  3.541375  5.285896 -1.015488   
                          APTV    1.804108  3.629080  5.454052 -0.571252   
...                                    ...       ...       ...       ...   
2023-12-31 00:00:00+00:00 MRNA    3.120799  4.847302  6.573804 -0.497320   
                          UBER    3.402687  4.859371  6.316055 -0.546214   
                          CRWD    3.168300  4.779138  6.389975 -0.554903   
                          PLTR    2.540490  4.605092  6.669694 -0.464274   
                          KVUE    2.333034  4.166136  5.999238 -1.401678   

                                      macd  
Date                      Ticker            
2016-02-29 00:00:00+00:00 AMAT   -0.002914  
                          AMGN   -0.550564  
                          AMT    -0.318322  
                          AMZN   -0.378774  
                          APTV   -0.326014  
...                                    ...  
2023-12-31 00:00:00+00:00 MRNA    0.508678  
                          UBER    1.532357  
                          CRWD    1.657772  
                          PLTR   -0.377428  
                          KVUE    2.116213  

[14155 rows x 8 columns]

In [123]:

def calculate_returns(df):
    outlier_cuteoff = 0.005
    lags = [1,2,3,6,9,12]

    for lag in lags:
        df[f'return_{lag}m'] = df['adj close'].pct_change(lag).pipe(lambda x:x.clip(lower=x.quantile(outlier_cuteoff),upper=x.quantile(1-outlier_cuteoff))).add(1).pow(1/lag).sub(1)
    
    return df

    
data = data.groupby(level=1, group_keys=False).apply(calculate_returns).dropna()
data
    

adj close  garman_klass_vol        rsi  \
Date                      Ticker                                            
2017-02-28 00:00:00+00:00 AMAT     33.276012         -0.002934  63.319270   
                          AMGN    141.052017         -0.018278  75.066041   
                          AMZN     42.251999          0.000079  59.641814   
                          AVGO     16.563255         -0.025305  64.951301   
                          AXP      71.944786         -0.004434  67.257353   
...                                      ...               ...        ...   
2023-12-31 00:00:00+00:00 ZTS     195.897491          0.000034  66.791321   
                          MRNA     99.449997          0.000363  65.939985   
                          UBER     61.570000          0.000375  64.522175   
                          CRWD    255.320007          0.000215  73.393415   
                          PLTR     17.170000          0.000326  44.949231   

                                    bb-low    bb-mid   bb-high       atr  \
Date                      Ticker                                           
2017-02-28 00:00:00+00:00 AMAT    2.915128  4.051850  5.188571 -1.099908   
                          AMGN    2.414501  3.879731  5.344961 -1.038652   
                          AMZN    2.406603  3.903004  5.399405 -1.308312   
                          AVGO    2.351784  3.957844  5.563904 -1.043369   
                          AXP     2.878728  4.143263  5.407797 -1.186299   
...                                    ...       ...       ...       ...   
2023-12-31 00:00:00+00:00 ZTS     2.874206  4.464028  6.053849  0.420520   
                          MRNA    3.120799  4.847302  6.573804 -0.497320   
                          UBER    3.402687  4.859371  6.316055 -0.546214   
                          CRWD    3.168300  4.779138  6.389975 -0.554903   
                          PLTR    2.540490  4.605092  6.669694 -0.464274   

                                      macd  return_1m  return_2m  return_3m  \
Date                      Ticker                                              
2017-02-28 00:00:00+00:00 AMAT    0.072613   0.060533   0.060945   0.040981   
                          AMGN    1.313750   0.134462   0.102588   0.072531   
                          AMZN    0.060737   0.026182   0.061563   0.040308   
                          AVGO    0.101560   0.057293   0.092358   0.075569   
                          AXP     0.218004   0.048180   0.041793   0.037282   
...                                    ...        ...        ...        ...   
2023-12-31 00:00:00+00:00 ZTS     2.164720   0.117168   0.121220   0.043775   
                          MRNA    0.508678   0.279923   0.144221  -0.012549   
                          UBER    1.532357   0.092054   0.192727   0.102136   
                          CRWD    1.657772   0.077345   0.201816   0.151137   
                          PLTR   -0.377428  -0.143641   0.077096   0.023804   

                                  return_6m  return_9m  return_12m  
Date                      Ticker                                    
2017-02-28 00:00:00+00:00 AMAT     0.033861   0.045875    0.057060  
                          AMGN     0.008527   0.014619    0.020342  
                          AMZN     0.015804   0.017514    0.036042  
                          AVGO     0.031721   0.036667    0.039864  
                          AXP      0.035409   0.023728    0.032524  
...                                     ...        ...         ...  
2023-12-31 00:00:00+00:00 ZTS      0.023770   0.019882    0.025896  
                          MRNA    -0.032826  -0.047138   -0.048072  
                          UBER     0.060957   0.075427    0.076054  
                          CRWD     0.096542   0.071393    0.076609  
                          PLTR     0.019072   0.081964    0.085428  

[11897 rows x 14 columns]